In [27]:
#https://query2.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/AAPL?symbol=AAPL"
#https://query2.finance.yahoo.com/v8/finance/chart/AAPL?period1=1704145844&period2=1723236317&interval=1mo

In [2]:
import sys
import os
import pandas as pd

cwd = os.getcwd()

sys.path.append(cwd.replace('notebooks','')+'DeltaStock')

In [2]:
from source.scraper import get_companyfacts
#https://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json
facts = get_companyfacts('CIK0001045810') #2016-07-31
facts.head()#

,finType,label,description,end,val,accn,fp,form,filed,frame,start,monthWindow,yearMonthDay
0,AcceleratedShareRepurchaseProgramAdjustment,"Accelerated Share Repurchase Program, Adjustment",The amount needed to adjust previously recorde...,2016-07-31,9000000.00,0001045810-16-000300,Q2,10-Q,2016-08-23,CY2016Q2,2016-05-02,3.0,20160731
1,AcceleratedShareRepurchaseProgramAdjustment,"Accelerated Share Repurchase Program, Adjustment",The amount needed to adjust previously recorde...,2016-10-30,9000000.00,0001045810-16-000353,Q3,10-Q,2016-11-22,NaN,2016-02-01,9.0,20161030
2,AcceleratedShareRepurchasesFinalPricePaidPerShare,"Accelerated Share Repurchases, Final Price Pai...",Final price paid per share for the purchase of...,2015-10-25,21.63,0001045810-15-000173,Q3,10-Q,2015-11-18,NaN,2015-01-26,9.0,20151025
3,AcceleratedShareRepurchasesFinalPricePaidPerShare,"Accelerated Share Repurchases, Final Price Pai...",Final price paid per share for the purchase of...,2016-07-31,42.06,0001045810-16-000300,Q2,10-Q,2016-08-23,NaN,2016-02-01,6.0,20160731
4,AcceleratedShareRepurchasesFinalPricePaidPerShare,"Accelerated Share Repurchases, Final Price Pai...",Final price paid per share for the purchase of...,2016-10-30,42.06,0001045810-16-000353,Q3,10-Q,2016-11-22,NaN,2016-02-01,9.0,20161030


In [3]:
from source.scraper import get_StockPrices

df = get_StockPrices('AAPL', startSelect='1986-01-02', endSelect='2022-01-02')
df.head()#11

,adjClose,ticker,quarter_meanADJclose,month_meanADJclose,yearMonthDay
0,0.076573,AAPL,0.084571,0.079429,19860102
1,0.077003,AAPL,0.084571,0.079429,19860103
2,0.076573,AAPL,0.084571,0.079429,19860106
3,0.079155,AAPL,0.084571,0.079429,19860107
4,0.078725,AAPL,0.084571,0.079429,19860108


In [8]:
from source.scraper import generate_CIK_TICKER

df = generate_CIK_TICKER()
df.head()

,cik_str,ticker,title,cik10D
0,320193,AAPL,Apple Inc.,0000320193
1,789019,MSFT,MICROSOFT CORP,0000789019
2,1045810,NVDA,NVIDIA CORP,0001045810
3,1652044,GOOGL,Alphabet Inc.,0001652044
4,1018724,AMZN,AMAZON COM INC,0001018724


In [4]:
from spark_deltalake.spark_operations import sparkDelta


spark = sparkDelta()

hadoop-azure is available
azure-storage is available


In [5]:
from ingestion import load_ticket_to_delta

#initialize_spy_ticker_sec()
#ingest_Facts_Fillings(sparkClass,'NVDA')
#load_spy500(sparkClass, 190, 190)
tickers = ['MMM' , 'AOS', 'ABT']
#get_fact_dataframes_dict('CIK0001045810', 'NVDA')
#get_spy500_batch_sec(batchSize=3)
df = load_ticket_to_delta(spark, batchSize=16, tickers = tickers)


In [16]:
import numpy as np

fillings_concat = pd.DataFrame()
facts_concat = pd.DataFrame()

for dic in df:
    fillings_concat = pd.concat([dic['SEC_filings'],fillings_concat ], axis = 0)
    facts_concat = pd.concat([dic['company_facts'],fillings_concat ], axis = 0)

In [18]:
fillings_concat['ticker'].value_counts()

MMM    1007
AOS    1002
ABT    1001
Name: ticker, dtype: int64

In [6]:
path = spark.blobPath + '/spy500'
spark.spark.sql(f'SELECT * FROM delta.`{path}`').show()

+------+--------------------+--------------------+--------------------+---------------------+----------+----------+-----------+
|Symbol|            Security|         GICS_Sector|   GICS_Sub_Industry|Headquarters_Location|Date_added|       CIK|    Founded|
+------+--------------------+--------------------+--------------------+---------------------+----------+----------+-----------+
|   MMM|                  3M|         Industrials|Industrial Conglo...| Saint Paul, Minne...|1957-03-04|0000066740|       1902|
|   AOS|         A. O. Smith|         Industrials|   Building Products| Milwaukee, Wisconsin|2017-07-26|0000091142|       1916|
|   ABT| Abbott Laboratories|         Health Care|Health Care Equip...| North Chicago, Il...|1957-03-04|0000001800|       1888|
|  ABBV|              AbbVie|         Health Care|       Biotechnology| North Chicago, Il...|2012-12-31|0001551152|2013 (1888)|
|   ACN|           Accenture|Information Techn...|IT Consulting & O...|      Dublin, Ireland|2011-07-06|

In [13]:
spy500 = spark.get_spark_dataframe(filename='spy500', toDataframe = True)
df[['Symbol', 'CIK']].to_numpy()

array([['MMM', '0000066740'],
       ['AOS', '0000091142'],
       ['ABT', '0000001800'],
       ...,
       ['ZBRA', '0000877212'],
       ['ZBH', '0001136869'],
       ['ZTS', '0001555280']], dtype=object)

In [2]:
from source.scraper import get_SEC_filings

ticker = 'AAPL'
filings = get_SEC_filings('CIK0001467373', 'ACN')
filings.head()#35

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription,fileURL,yearMonthDay,ticker
0,0001467373-24-000248,2024-08-16,2024-08-15,2024-08-16T16:17:17.000Z,,4,,,,4750,0,0,xslF345X05/wk-form4_1723839430.xml,FORM 4,https://www.sec.gov/Archives/edgar/data/000146...,20240815,ACN
1,0001467373-24-000247,2024-08-16,2024-08-15,2024-08-16T16:17:06.000Z,,4,,,,5688,0,0,xslF345X05/wk-form4_1723839420.xml,FORM 4,https://www.sec.gov/Archives/edgar/data/000146...,20240815,ACN
2,0001467373-24-000246,2024-08-16,2024-08-15,2024-08-16T16:16:55.000Z,,4,,,,4802,0,0,xslF345X05/wk-form4_1723839409.xml,FORM 4,https://www.sec.gov/Archives/edgar/data/000146...,20240815,ACN
3,0001467373-24-000245,2024-08-16,2024-08-15,2024-08-16T16:16:45.000Z,,4,,,,4696,0,0,xslF345X05/wk-form4_1723839398.xml,FORM 4,https://www.sec.gov/Archives/edgar/data/000146...,20240815,ACN
4,0001467373-24-000244,2024-08-16,2024-08-15,2024-08-16T16:16:33.000Z,,4,,,,5596,0,0,xslF345X05/wk-form4_1723839387.xml,FORM 4,https://www.sec.gov/Archives/edgar/data/000146...,20240815,ACN


In [3]:
from ingestion import get_time_fact_dataframes, get_time_fact_dataframes_async, get_spy500_batch_sec

res = get_spy500_batch_sec('CIK0001045810', 'NVDA')
#normal - 45 sek
#async - 45 sek
#concurrent - 
res

ImportError: cannot import name 'get_time_fact_dataframes' from 'ingestion' (c:\programowanie\DCFmodeling\DeltaStock\ingestion.py)

In [13]:
pydf = spark.get_spark_dataframe('SEC_filings', True)
pydf.show()

+--------------------+----------+----------+--------------------+---+----+----------+----------+-------------------+------+------+------------+--------------------+---------------------+--------------------+------------+------+
|     accessionNumber|filingDate|reportDate|  acceptanceDateTime|act|form|fileNumber|filmNumber|              items|  size|isXBRL|isInlineXBRL|     primaryDocument|primaryDocDescription|             fileURL|yearMonthDay|ticker|
+--------------------+----------+----------+--------------------+---+----+----------+----------+-------------------+------+------+------------+--------------------+---------------------+--------------------+------------+------+
|0001965301-23-000185|2023-12-22|          |2023-12-22T12:58:...| 33| 144| 000-15175| 231508590|                   |  4840|     0|           0|xsl144X01/primary...|                     |https://www.sec.g...|    20231222|  ADBE|
|0000796343-23-000258|2023-12-19|2023-12-15|2023-12-19T18:02:...|   |   4|          |   

In [14]:
pydf = spark.get_spark_dataframe('company_facts', True)
pydf.show()

+--------------------+--------------------+--------------------+----------+----------+--------------------+------+---+----+----------+---------+--------------------+----------+--------------------+--------+-----------+------------+
|             finType|               label|         description|       end|       val|                accn|    fy| fp|form|     filed|    frame|           endFormat|     start|         startFormat|diffDate|monthWindow|yearMonthDay|
+--------------------+--------------------+--------------------+----------+----------+--------------------+------+---+----+----------+---------+--------------------+----------+--------------------+--------+-----------+------------+
|IncreaseDecreaseI...|Increase (Decreas...|The increase (dec...|2020-09-30|    -4.8E7|0000066740-21-000013|2021.0| Q3|10-Q|2021-10-26|      NaN|java.util.Gregori...|2020-01-01|java.util.Gregori...|   273.0|        9.0|    20200930|
|IncreaseDecreaseI...|Increase (Decreas...|The increase (dec...|2021-09-

In [15]:
ticker = 'AOS'
df = spark.get_min_max_fillings_date(ticker)
df

,ticker,max_time,min_time
0,AOS,2024-08-16,2012-12-31


In [ ]:
spark.sparkStop()

In [16]:
from source.scraper import get_spy500_formWiki


wiki = get_spy500_formWiki()
wiki.head()

,Symbol,Security,GICS_Sector,GICS_Sub_Industry,Headquarters_Location,Date_added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,0000066740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,0000001800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989


In [5]:
facts.loc[:, 'yearMonthDay']

0        20160731
1        20161030
2        20151025
3        20160731
4        20161030
           ...   
23308    20230129
23309    20230430
23310    20240128
23311    20240428
23312    20240428
Name: yearMonthDay, Length: 23313, dtype: object

In [1]:
import numpy as np
ticketList = np.array([["MMM","CIK0000066740"],
                    ["AOS","CIK0000091142"],
                    ["ABT","CIK0000001800"] ])

In [2]:
ticketList[:,1]

array(['CIK0000066740', 'CIK0000091142', 'CIK0000001800'], dtype='<U13')

In [ ]:
"""
https://www.sec.gov/ix?doc=/Archives/edgar/data/0000320193/000114036124024352/ef20028273_8k.htm


https://data.sec.gov/api/xbrl/companyconcept/CIK0000320193/us-gaap/AccountsPayableCurrent.json - info about reports

https://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json

/0001140361|24|024352/
accessionNumber - 0001140361-24-024352

"""

In [69]:
import numpy as np
ticketList = np.array([["MMM","CIK0000066740"],
                       ["AOS1","CIK0000091142"],
                       ["AOS11","CIK0000091142"],
                       ["AOS111","CIK0000091142"],
                       ["AOS2","CIK0000091142"],
                    ["AOS3","CIK0000091142"],
                    ["ABT","CIK0000001800"] ])
    
wholeDataInBatch = []
bachSize = 1
start_step = 0
for end_step in range(bachSize, len(ticketList), bachSize):
    ticker = list(ticketList[start_step:end_step,0])
    ciks = list(ticketList[start_step:end_step,1])
    print(ticker)
    start_step = end_step
    
if start_step != len(ticketList):
    ticker = list(ticketList[start_step:,0])
    ciks = list(ticketList[start_step:,1])
    print(ticker)

['MMM']
['AOS1']
['AOS11']
['AOS111']
['AOS2']
['AOS3']
['ABT']
